In [30]:
pip install boxdetect


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [31]:
from PIL import Image
import cv2
import os
import re
#from pdf2image import convert_from_path
from wand.image import Image as WandImage
from PyPDF2 import PdfReader
from boxdetect import config
from boxdetect.pipelines import get_checkboxes
from colorama import Fore, Back, Style
import os
import os.path
import numpy as np
import pandas as pd
import fitz  # PyMuPDF
import io
from PIL import Image, ImageDraw, ImageFont
from PIL import Image, ImageFilter
from xlsxwriter import workbook
import pandas as pd
from pandas import ExcelWriter
import openpyxl as xl
import glob
import os
import openpyxl
from numpy import asarray
from openpyxl import load_workbook
import pytesseract
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import random
from itertools import groupby
from operator import itemgetter
import time

pytesseract.pytesseract.tesseract_cmd = r"C:\Users\maxik\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
#pytesseract.pytesseract.tesseract_cmd = 'C:\\Users\\maxik\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe'

In [32]:

# Load your image (replace 'your_image.png' with th

In [33]:
def increase_resolution(input_path, output_path, scale_factor):
    with WandImage(filename=input_path) as img:
        # Set resolution
        img.resolution = (img.width * scale_factor, img.height * scale_factor)
        # Save the high-resolution image
        img.save(filename=output_path)
def sharpen_image(input_path, output_path, factor=2):
    # Open the image
    image = Image.open(input_path)

    # Apply the sharpen filter
    sharpened_image = image.filter(ImageFilter.UnsharpMask(radius=2, percent=factor, threshold=1))

    # Save the sharpened image
    sharpened_image.save(output_path)
def getboxcord(numpyarr, n=6):
    verticle = np.sum(numpyarr, 0).tolist()
    horrazontal = np.sum(numpyarr, 1).tolist()
    v =verticle
    h= horrazontal
    verticle.sort()
    horrazontal.sort()
    verticleDuplicates = [verticle for verticle in verticle if verticle.count(verticle) > 1]
    vunique_duplicates = list(set(verticleDuplicates))
    horrazontalDuplicates = [horrazontal for horrazontal in horrazontal if horrazontal.count(horrazontal) > 1]
    hunique_duplicates = list(set(horrazontalDuplicates))
    topnv = vunique_duplicates[0:n]
    topnh = hunique_duplicates[0:n]
    x1=[]
    y1=[]
    x2=[]
    y2=[]
    x1[0:n].append((v.index(topnv[0:n]), h.index(topnh[0:n])))
    y1[0:n].append((v.index(topnv[0:n]) + topnv[0:n], h.index(topnh[0:n])))
    x2[0:n].append((v.index(topnv[0:n]), h.index(topnh[0:n])+ topnh[0:n]))
    y2[0:n].append((v.index(topnv[0:n]) + topnv[0:n], h.index(topnh[0:n])+ topnh[0:n]))
    print(y2)
    print(list(v.index(topnv[0:n]) + topnv[0:n], h.index(topnh[0:n])+ topnh[0:n]))
    lefts = []
    rights = []
    lower = []
    upper = []

def crop_item(dir, Le_Up_Ri_Lo, root_dir, left=0, upper=120, right=2000, lower=1000):
    image = Image.open(dir)  # Replace with the path to your image
        
    # except:
    #     print(dir+" erro failed to read")
    #     return 1
    # Define the coordinates of the rectangle you want to crop (left, upper, right, lower)
    list_filename = dir.split('.')
    filename=list_filename[0]+".png"
    #^ nuron network to identify cordinates of a data frame rectangle, 
    # Crop the image
    getboxcord(np.array(image))
    if((image.width == right-left and image.width == lower-upper) or "0" in filename):
        return filename
    cropped_image = image.crop((left, upper, right, lower))
    filename=list_filename[0]+"0"+".png"
    # Save the cropped image
    cropped_image.save(filename)
    return filename
def detect_horizontal_lines(img_array, threshold=0.95):
    # Calculate the sum of pixel values along each row
    row_sums = np.sum(img_array, axis=1)

    # Detect horizontal lines by finding rows with pixel sums exceeding the threshold
    horizontal_lines = np.where(row_sums >= threshold * img_array.shape[1])[0]
    
    return horizontal_lines


def pdf_to_image_array(pdf_path, output_image_path, page_number=0):
    output_filename = output_image_path.split(".")
    output_filename = output_filename[0] + "v6"+".png"
    print(output_filename)
    pdf_document = fitz.open(pdf_path)
    page = pdf_document[page_number]
    zoom = 4    # zoom factor
    mat = fitz.Matrix(zoom, zoom)
    image = page.get_pixmap(matrix = mat)
    
    
    # Save the image to the specified path

    image.save(output_filename, "PNG")
    # Close the PDF document
    #pdf_document.close()

In [34]:
def flashfill(x_seed, y_seed, img_path, color=(255, 255, 255, 255)):
    img = Image.open(img_path)
    img1 = img.convert("RGB") 
    # Create an ImageDraw object
    # Perform flood fill
    ImageDraw.floodfill(img,(x_seed, y_seed), color)
    # cv2.imshow('flash', asarray(img))
    # # Break the loop if 'q' key is pressed
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     cv2.destroyAllWindows()
    return np.array(img)
def relative_flushFlood(img_path,relative_big_lines):
    output_path = img_path.split()
    output_path1 = output_path[0]+str(random.randint(0, 100)) + ".png"
    img = Image.open(img_path)
    i=0
    while(i<= 20):
        x=random.randint(0, img.width)
        try:
            y=relative_big_lines[random.randint(0, len(relative_big_lines))]
        except:
            y=img.height
        try:
            if(img.getpixel((x,y)) == (0,0,0,0)):
                pass
            else:
                modimg = Image.fromarray(flashfill(x, y, img_path), 'RGB')
                modimg.save(img_path)
                img=modimg
        except:
            pass
        i+=1
    return np.array(img)
def precent(img_array):
    viritile_lines = []
    column_sums = []
    column_sums = (np.sum(img_array, axis=0))
    virticle_counter = 0
    column_sums=list(column_sums)
    try:
        column_sums = [item for arr in column_sums for item in arr.tolist()]
    except:
        pass
    #print(img_array)
    #print(column_sums)
    for i in column_sums:
        if (i == min(column_sums)):
            indices = np.where(column_sums == i)[0]
            viritile_lines.extend(indices)
            virticle_counter+=1
    try:
        precentage_virticle_white = (virticle_counter/len(column_sums)) *100
    except:
        precentage_virticle_white = 100
    return precentage_virticle_white
def margin_detect_verticle_lines(img_array, threshold=1.0):
    # Calculate column sums
    column_sums = np.sum(img_array, axis=0)
    
    # Sort indices based on column sums
    sorted_indices = np.argsort(column_sums)
    
    # Threshold for significant vertical lines
    threshold_value = threshold * max(column_sums)
    
    # Initialize lists for detected lines
    vertical_lines = []
    large_lines = []
    
    for i in column_sums:
        if i >= threshold_value:
            indices = np.where(column_sums == i)[0]
            vertical_lines.extend(indices)
            
            # Check for large lines (you can adjust this condition)
            if len(indices) > 10:  # Example: consider lines wider than 10 pixels
                large_lines.extend(indices)
    
    # Calculate percentage of significant vertical white space
    try:
        percentage_vertical_white = (len(vertical_lines) / len(column_sums)) * 100
    except ZeroDivisionError:
        percentage_vertical_white = 0
    vertical_lines=set(vertical_lines)
    vertical_lines=list(vertical_lines)
    vertical_lines.sort()
    relative_verticle_lines = []
    for k, g in groupby(enumerate(vertical_lines), lambda ix: ix[0] - ix[1]):
        consecutive_numbers = list(map(itemgetter(1), g))
        #print(consecutive_numbers)
        relative_verticle_lines.append(int(sum(consecutive_numbers)/len(consecutive_numbers)))
    return relative_verticle_lines, large_lines, percentage_vertical_white
def is_image_black(img):
    return not img.getbbox()
def smart_crop_relative(image_path, output_path):
    list_filename = image_path.split('.')
    list_filename2=list_filename[0]+"cropped\\"
    os.makedirs(list_filename2, exist_ok=True)
    img1= Image.open(image_path).convert('L')
    img = img1
    img_array = np.array(img)
    (relative_verticle_lines, relative_big_lines, precentage) = margin_detect_verticle_lines(img_array)
    img = Image.fromarray(relative_flushFlood(image_path, relative_big_lines))
    # relative_verticle_lines=set(relative_verticle_lines)
    # relative_verticle_lines=list(relative_verticle_lines)
    # relative_verticle_lines.sort()
    print(relative_verticle_lines)
    leftv = []
    rightv = []
    if(len(relative_verticle_lines) > 0):
        rr = 0
        ll = 0
        #relative_verticle_lines.sort()
        for rr in relative_verticle_lines:
            up = 0
            bottom = 1000
                #img = img.resize((right, img.height))
            if(ll>=rr):
                ll=0
                if(ll>=rr):
                    rr=20
            if(rr>=img.width):
                new_image = Image.new(img.mode, (img.width+(rr-img.width), img.height))
                new_image.paste(img, (0, 0))
                img=new_image
                # else:
                #     new_image = Image.new(img.mode, (img.width+(img.width-rr), img.height))

    # Paste the original image onto the new canvas
            cropped_image = img.crop((ll, up, rr, bottom))
            numpyarrCrppedimg=np.array(cropped_image)
            value = precent(numpyarrCrppedimg)
            value= int(value)
            #print(":value::", value)
            if(rr >= cropped_image.width):
                new_image = Image.new(img.mode, (cropped_image.width+(rr-cropped_image.width), cropped_image.height))

                new_image.paste(cropped_image, (0, 0))
                cropped_image=new_image
            if(value<=90):
                filename=list_filename2+str(rr)+".png"
                cropped_image.save(filename)
                cv2.imshow
                #display(cropped_image)
        #     try:
        #         cv2.imshow('Croppped img', asarray(cropped_image))
        #     except:
        #         pass
        # # Break the loop if 'q' key is pressed
        #     if cv2.waitKey(1) & 0xFF == ord('q'):
        #         break
        #     try:
        #         cv2.imshow('img', asarray(img))
        #     except:
        #         pass
        # # Break the loop if 'q' key is pressed
        #     if cv2.waitKey(1) & 0xFF == ord('q'):
        #         break
            if(is_image_black(cropped_image)):
                break
            rightv.append(rr)
    cv2.destroyAllWindows()
    directory = list_filename[0]+"cropped\\"
    file_list_cropped = os.listdir(directory)
    file_list_cropped2= []
    for i in file_list_cropped:
        file_list_cropped2.append(directory+i)
    file_list_cropped2.sort()
    #print(file_list_cropped2)
    for i in range(len(file_list_cropped2)):
        #open i and i+1
        try:
            imgMain=Image.open(file_list_cropped2[i]).convert('L')
            heavyvar = 0
            if(i<=1):
                heavyvar=rightv[0]
            else:
                heavyvar=rightv[rightv.index(imgMain.width)-1]
            while(int(heavyvar)>imgMain.width and heavyvar>0):
                heavyvar-=1
            cropedMain = imgMain.crop((int(heavyvar), 0, imgMain.width, 1000))
            cropedMain.save(file_list_cropped2[i])
            #print(file_list_cropped2[i])
            imgMain.close()
        except:
            #print(file_list_cropped2[i])
            pass
    return directory


In [35]:

def PrecentageMatch(string1, string2):
    precetage_count = 0
    for i in range(len(string1)):
        #print(string1)
        #print(string2)
        try:
            if(string1[i] == string2[i]):
                precetage_count=+1
        except:
            precetage_count= precetage_count

    if(precetage_count!=0):
        return precetage_count/len(string1)*100
    return 0
def Compare(string1, string2):
    '''
    Compares two lists in a loop. Once the lists do not
    match retures charater of lists that do not match.
    '''

    for i in range(len(string1)):
        try:
            if(string1[i] != string2[i]):
                return string1
        except:
            pass
    return "error"
def match_join(string1, string2):
    #print(string1)
    #print(string2)
    try:
        string1_list = string1.split()
    except:
        print(" ")
        #print("except_string1")
    try:
        string2_list = string2.split()
    except:
        #print("excelption_string2")
        print(" ")
    bitmap_append = []
    output_string = " "
    for i in range(len(string1_list)):
        try:
            if(string1_list[i] == string2_list[i]):
                bitmap_append.append(1)
            else:
                bitmap_append.append(0)
        except:
            bitmap_append.append(0)
    for j in range(len(bitmap_append)):
        try:
            if(bitmap_append[j]==1):
                output_string = output_string+string2_list[j]
        except:
            pass
    return output_string
def create_scv(dir, filename_out, data_list, dir_outs):
    if(len(data_list) <=1):
        return 1
    filename_outX = dir_outs+".xlsx"
    df = pd.DataFrame(data_list)
    print(filename_outX)
    display(df)
    df.to_excel(filename_outX)
def combineONEXL(path, output):
    #path = r'C:\path\to\your\directory'
    print(path)
# Get a list of all .xlsx files in the directory
    file_list_cropped = os.listdir(path)
    file_list_cropped2= []
    for i in file_list_cropped:
        file_list_cropped2.append(path+i)
    files_xls= file_list_cropped2
    print(files_xls)
    # Initialize an empty DataFrame
    combined_df = pd.DataFrame()

    # Loop through each file and append its data to the combined DataFrame
    data = pd.DataFrame()
    for file in files_xls:
        data = pd.read_excel(file, sheet_name='Sheet1')  # Change 'Sheet1' to your sheet name
        combined_df = pd.concat([combined_df, data], axis=1)
        display(combined_df)
        display(data)

    # Now you have a single DataFrame containing data from all Excel files
    combined_df.to_excel(output)
    return path
def pytesseractRead(dir, output_dir, sizeofDataframeROWSCOLS):
    sharpen_image(dir, dir, 2)
    print(dir)
    image = Image.open(dir)  # Replace with the path to your image

# Use pytesseract with custom configuration
    text_colums = pytesseract.image_to_string(image, timeout=2)
    custom_config = r'--psm 6'  # Horizontal text mode
    text_lines = pytesseract.image_to_string(image, config=custom_config, timeout=2)
    for i in text_lines:
        if(i=="\n" and (text_lines[text_lines.index(i)-1]=="\n" or text_lines[text_lines.index(i)-1]==" ")):
            text_lines[text_lines.index(i)]=""
    for i in text_colums:
        if(i=="\n" and (text_colums[text_colums.index(i)-1]=="\n" or text_colums[text_colums.index(i)-1]==" ")):
            text_colums[text_colums.index(i)]=""

    #creats lists
    text_rows_inline = text_lines.split("\n")
    text_colums_nline = text_colums.split("\n")
    df = pd.DataFrame(text_colums_nline)
    #print(datalistdf)
    filename = output_dir.split("\\")
    create_scv(dir, filename[-1], df, output_dir)
    return dir


### drg 09gyhrg 9uhregpiuhre gpiuh rgepiuh reg piuh rpiuhregpiuhgfreiuhregiuhrgeiuhrg

In [36]:

def itr_dir(dir, extension, cvsDirout, jdg_dir=r"C:\Users\maxik\Documents\Github\Nuron-text-recognition\WNB-PUCD6-DH-1432-jdg"):  # Replace with your desired file extension
    # List all files in the directory

    # Filter files by the specified extension
    # Print the filtered file list
    try:
        file_list = os.listdir(jdg_dir)
        filtered_files = [file for file in file_list if file.endswith(".png")]
    except:
        file_list = []
        filtered_files = []
    if(len(file_list) != 0 and len(filtered_files)!=0):
        file_list = os.listdir(jdg_dir)
        filtered_files = [file for file in file_list if file.endswith(".png")]
        if(len(filtered_files) == 0):
            return 1
        dir_oututcvs = cvsDirout.split("\\")
        print(dir_oututcvs)
        dir_out2 = dir_oututcvs[0:len(dir_oututcvs)]
        print(dir_out2)
        dir_outs = dir_out2[0]
        for w in dir_out2:
            print(w)
            if(w!=dir_out2[0]):
                dir_outs = dir_outs+"\\"+w
            print(dir_outs)
        print("hello!")
        for file in filtered_files:
            dir_fle = ""
            print("hello")
            dir_file = jdg_dir+'\\'+file
            list_filenames = file.split('.')
            print('extrancting')
            dir_20=crop_item(dir_file, 0, jdg_dir)
            if(dir_20==1):
                print("crash")
                continue
            ret =smart_crop_relative(dir_20,dir_20)
            if(ret!= 1): 
                cropped_images = ret
                print(cropped_images)
                file_list_cropped = os.listdir(cropped_images)
                file_list_cropped2= []
                for d in file_list_cropped:
                    file_list_cropped2.append(cropped_images+d)      
                for q in file_list_cropped2:
                    output_dir = q
                    if(output_dir != 1):
                        print("crop")
                        print(output_dir)
                        filename = output_dir.split("\\")
                        files = filename[-1].split(".")
                        dir_outcvs = cvsDirout+"\\"+file+"\\"
                        print("sss"+dir_outcvs)
                        os.makedirs(dir_outcvs, exist_ok=True)
                        pytesseractRead(output_dir, dir_outcvs+(files[0].split("."))[0], (0,0))
                        return 0
                print("?????????????????????????????????????????????????????????????")
                combineONEXL(dir_outcvs, dir_outcvs+(file[0].split("."))[0]+".xlsx") 
            else:
                print("error")
        return 3
    else:
        file_list = os.listdir(dir)
        filtered_files = [file for file in file_list if file.endswith(extension)]
        os.makedirs(jdg_dir, exist_ok=True)
        for file in filtered_files:
            dir_file = ""
            dir_file = dir+'\\'+file
            list_filenames = file.split('.')
            dir_filejpg = ""
            dir_filejpg = jdg_dir+'\\'+file
            if(list_filenames[-1] == "pdf"):
                    #pdf_to_jpg(dir_file, dir)
                    #extract_text_from_column(dir_file, 0)
                pdf_to_image_array(dir_file, dir_filejpg)
                #print(dir_file)
        return 2
input_dir=r"C:\Users\maxik\Documents\GitHub\Nuron-text-recognition\WNB-PUCD6-DH-1432"
#input_dir=r"C:\Users\maxik\Documents\GitHub\Nuron-text-recognition\support-right"
jdg_dir = r"C:\Users\maxik\Documents\Github\Nuron-text-recognition\WNB-PUCD6-DH-1432-jdg"
cvsDirout = r"C:\Users\maxik\Documents\Github\Nuron-text-recognition\WNB-PUCD6-DH-1432-CVS"
os.makedirs(cvsDirout, exist_ok=True)
if(itr_dir(input_dir, ".pdf", cvsDirout) == 2):
    itr_dir(input_dir, ".pdf", cvsDirout)

['C:', 'Users', 'maxik', 'Documents', 'Github', 'Nuron-text-recognition', 'WNB-PUCD6-DH-1432-CVS']
['C:', 'Users', 'maxik', 'Documents', 'Github', 'Nuron-text-recognition', 'WNB-PUCD6-DH-1432-CVS']
C:
C:
Users
C:\Users
maxik
C:\Users\maxik
Documents
C:\Users\maxik\Documents
Github
C:\Users\maxik\Documents\Github
Nuron-text-recognition
C:\Users\maxik\Documents\Github\Nuron-text-recognition
WNB-PUCD6-DH-1432-CVS
C:\Users\maxik\Documents\Github\Nuron-text-recognition\WNB-PUCD6-DH-1432-CVS
hello!
hello
extrancting


ValueError: [] is not in list